In [ ]:
%%capture
!pip install nlpaug
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install tensorflow_probability==0.12.2
# add bert base to do the synonymaug replacement
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSeq2SeqLM
import torch
from google.colab import drive
import os
import pandas as pd
import json
import nlpaug.augmenter.word as naw
import random
import nltk
import gensim.downloader as api
from nltk.tokenize import word_tokenize

from nltk.metrics.distance import edit_distance
from tqdm import tqdm
from transformers import pipeline
import datasets
import numpy as np
from datasets import Dataset
from datasets import load_dataset

In [ ]:
# Mount Google Drive to access the files
drive.mount('/content/drive')

# Define the path to save the CSV files
path = '/content/drive/My Drive/dsml/1_project/Dataset/second'
os.chdir(path)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Mounted at /content/drive


In [ ]:
data = load_dataset('codeparrot/xlcost-text-to-code', "Python-program-level", split='train')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
# Followed by separate the code and nl
nl_data = []
code_data = []
for text in data:
  nl_data.append(text['text'])
  code_data.append(text['code'])

## Use starcoder

In [ ]:
!huggingface-cli login
device = "cuda" # for GPU usage or "cpu" for CPU usage

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoderbase-1b")
model = AutoModelForCausalLM.from_pretrained("bigcode/starcoderbase-1b").to(device)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [ ]:
for i in range(len(nl_data)):
  inputs = tokenizer.encode("What is the purpose of the following code?"+code_data[i], return_tensors="pt").to(device)
  outputs = model.generate(inputs)
  print("augment data =",tokenizer.decode(outputs[0]))
  print("original data=",nl_data[i])
  print('---------------')

augment data = What is the purpose of the following code?def maxPresum ( a, b ) : NEW_LINE INDENT X = max ( a [ 0 ], 0 ) NEW_LINE for i in range ( 1, len ( a ) ) : NEW_LINE INDENT a [ i ] += a [ i - 1 ] NEW_LINE X = max ( X, a [ i ] ) NEW_LINE DEDENT Y = max ( b [ 0 ], 0 ) NEW_LINE for i in range ( 1, len ( b ) ) : NEW_LINE INDENT b [ i ] += b [ i - 1 ] NEW_LINE Y = max ( Y, b [ i ] ) NEW_LINE DEDENT return X + Y NEW_LINE DEDENT A = [ 2, - 1, 4, - 5 ] NEW_LINE B = [ 4, - 3, 12, 4, - 3 ] NEW_LINE print ( maxPresum ( A, B ) ) NEW_LINE 
original data= Maximum Prefix Sum possible by merging two given arrays | Python3 implementation of the above approach ; Stores the maximum prefix sum of the array A [ ] ; Traverse the array A [ ] ; Stores the maximum prefix sum of the array B [ ] ; Traverse the array B [ ] ; Driver code
---------------
augment data = What is the purpose of the following code?import math NEW_LINE def sumOfTwoCubes ( n ) : NEW_LINE INDENT lo = 1 NEW_LINE hi = round ( math. p

KeyboardInterrupt: ignored

##Use lanuage model

### Replacing words with synonyms by bert language model

In [ ]:
# Replacing words with synonyms by language model
# Import the pipeline from huggingface
unmasker = pipeline('fill-mask', model='bert-base-cased',device=0, batch_size = 32)

# This function encapsulates the text augmentation strategy.
import re  # needed for checking if a word is a digit
def synonyms_aug(input_text):
    orig_text_list = input_text.split()

    # Calculate the target number of words to replace
    num_replace = round(len(orig_text_list) * 0.3)
    replaced_indices = []
    digit_indices = [i for i in range(len(orig_text_list)) if re.match(r'^\d+$', orig_text_list[i])]

    for _ in range(num_replace):
        # Random index where we want to replace the word
        rand_idx = random.choice([i for i in range(1, len(orig_text_list)-1) if i not in replaced_indices and i not in digit_indices])

        # Store the original word for comparation
        orig_word = orig_text_list[rand_idx]
        new_text_list = orig_text_list.copy()
        new_text_list[rand_idx] = '[MASK]'
        new_mask_sent = ' '.join(new_text_list)
        try:
          augmented_text_list = unmasker(new_mask_sent)

          # To assert the new word is different
          for res in augmented_text_list:
              if res['token_str'] != orig_word:
                  augmented_text = res['sequence']
                  orig_text_list = augmented_text.split()
                  replaced_indices.append(rand_idx)
                  break
        except RuntimeError:
          return input_text

    return ' '.join(orig_text_list)


n = 2 # Augment for three times
augment_data = []

for i in tqdm(range(len(nl_data))):
    for _ in range(n): # Repeat n times
        augmented_text = synonyms_aug(nl_data[i])
        augment_data.append({'code': code_data[i], 'text': augmented_text})

  0%|          | 0/9263 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  0%|          | 32/9263 [00:44<3:32:56,  1.38s/it]


KeyboardInterrupt: ignored

In [ ]:
path_1 = '/content/drive/My Drive/dsml/1_project/Dataset/second/1_bert_sr_n2.json'

# write each dict to a new line
with open(path_1, 'w') as outfile:
    for entry in augment_data:
        json.dump(entry, outfile)
        outfile.write('\n')  # Add newline cause PyJSON does not

### Replacing words with synonyms by word2vec language model

In [ ]:
### Replacing words with synonyms by word2vec language model
nltk.download('punkt')

# Load Google's pre-trained Word2Vec model.
model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
import re
def replace_synonyms(sentence):

  words = word_tokenize(sentence)
  code_words = re.findall(r"`([^`]*)`", sentence)
  index_len = round(len(words)* 0.3)
  rand_index = np.random.choice(len(words), size=index_len, replace = False)

  for i in rand_index:
    if words[i] not in code_words and not words[i].isnumeric() and words[i] in model.key_to_index:
        # Get the most similar words
        similar_words = model.most_similar(positive=[words[i]], topn=1)
        # Select one of the most similar words
        replacement_word = random.choice(similar_words)[0]
        words[i] = replacement_word
  return ' '.join(words)

n = 2 # Augment for three times
augment_data_w = []

for i in tqdm(range(len(nl_data))):
    for _ in range(n): # Repeat n times
        augmented_text_w = replace_synonyms(nl_data[i])
        augment_data_w.append({'code': code_data[i], 'text': augmented_text_w})

100%|██████████| 9263/9263 [17:45:27<00:00,  6.90s/it]


In [ ]:
path_2 = '/content/drive/My Drive/dsml/1_project/Dataset/second/2_word2vec_sr_0.3_2.json'

# write each dict to a new line
with open(path_2, 'w') as outfile:
    for entry in augment_data_w:
        json.dump(entry, outfile)
        outfile.write('\n')  # Add newline cause PyJSON does not

### Back translation using language model

#### French

In [ ]:
import torch
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def back_translate(phrases, source_lang='en', target_lang='fr', batch_size=32):
    # Load the MarianMT tokenizer and model for translation from source language to target language
    model_name = f'Helsinki-NLP/opus-mt-{source_lang}-{target_lang}'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)
    model.eval()

    # Now, load the MarianMT tokenizer and model for translation from target language back to source language
    model_name_back = f'Helsinki-NLP/opus-mt-{target_lang}-{source_lang}'
    tokenizer_back = AutoTokenizer.from_pretrained(model_name_back)
    model_back = AutoModelForSeq2SeqLM.from_pretrained(model_name_back, torch_dtype=torch.float16).to(device)
    model_back.eval()

    back_translated_phrases = []

    # Process in batches
    for batch_start in range(0, len(phrases), batch_size):
        batch_phrases = phrases[batch_start: batch_start+batch_size]

        # Tokenize the batch of phrases
        inputs = tokenizer(batch_phrases, return_tensors="pt", padding=True, truncation=True, max_length=400).to(device)

        # Translate the batch of phrases to the target language
        outputs = model.generate(**inputs, max_length=256, eos_token_id=tokenizer.eos_token_id)
        translated_phrases = tokenizer.batch_decode(outputs, skip_special_tokens=True)

        # Tokenize the translated batch of phrases
        inputs_back = tokenizer_back(translated_phrases, return_tensors="pt", padding=True, truncation=True, max_length=400).to(device)

        # Translate the batch of phrases back to the source language
        outputs_back = model_back.generate(**inputs_back, max_length=256, eos_token_id=tokenizer.eos_token_id)
        back_translated_batch = tokenizer_back.batch_decode(outputs_back, skip_special_tokens=True)
        back_translated_phrases.extend(back_translated_batch)

    return back_translated_phrases

def fix_repeated_sequences(original, back_translated, threshold=4):
    # This regular expression pattern finds any sequence of non-whitespace characters that is repeated at least threshold times.
    pattern = r'(\S+)(?:\s*\1){' + str(threshold - 1) + ',}'

    if re.search(pattern, back_translated):  # If any repeated sequences are found
        return original  # replace back-translated string with original
    else:
        return back_translated

In [ ]:
n = 1  # Augment
batch_size = 32
augment_data_btf = []

for i in tqdm(range(0, len(nl_data), batch_size)):
    for _ in range(n):  # Repeat n times
        batch = nl_data[i : i+batch_size]
        back_translated_batch = back_translate(batch)

        for idx, back_translated_text in enumerate(back_translated_batch):
            fixed_text = fix_repeated_sequences(batch[idx], back_translated_text)
            augment_data_btf.append({'code': code_data[i+idx], 'text': fixed_text})

In [ ]:
path_f = '/content/drive/My Drive/dsml/1_project/Dataset/second/3_hels_btf.json'

# write each dict to a new line
with open(path_f, 'w') as outfile:
    for entry in augment_data_btf:
        json.dump(entry, outfile)
        outfile.write('\n')  # Add newline cause PyJSON does not

#### German

In [ ]:
! pip install sacremoses
n = 1  # Augment
batch_size = 32
augment_data_btg = []

for i in tqdm(range(0, len(nl_data), batch_size)):
    for _ in range(n):  # Repeat n times
        batch = nl_data[i : i+batch_size]
        back_translated_batch = back_translate(batch, 'en', 'de')
        for idx, back_translated_text in enumerate(back_translated_batch):
            fixed_text = fix_repeated_sequences(batch[idx], back_translated_text)
            augment_data_btg.append({'code': code_data[i+idx], 'text': fixed_text})

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895239 sha256=d1b97eee36c5537f8b7688c1d683b307eb7a1588665ba036c3ff8a3ea6cb787b
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


100%|██████████| 290/290 [2:04:02<00:00, 25.66s/it]


In [ ]:
path_g = '/content/drive/My Drive/dsml/1_project/Dataset/second/3_hels_btg.json'

# write each dict to a new line
with open(path_g, 'w') as outfile:
    for entry in augment_data_btg:
        json.dump(entry, outfile)
        outfile.write('\n')  # Add newline cause PyJSON does not

#### Spanish

In [ ]:
n = 1  # Augment
batch_size = 32
augment_data_bts = []

for i in tqdm(range(0, len(nl_data), batch_size)):
    for _ in range(n):  # Repeat n times
        batch = nl_data[i : i+batch_size]
        back_translated_batch = back_translate(batch, 'en', 'es')

        for idx, back_translated_text in enumerate(back_translated_batch):
            fixed_text = fix_repeated_sequences(batch[idx], back_translated_text)
            augment_data_bts.append({'code': code_data[i+idx], 'text': fixed_text})

 91%|█████████ | 264/290 [1:59:56<11:43, 27.07s/it]

In [ ]:
path_s = '/content/drive/My Drive/dsml/1_project/Dataset/second/3_hels_bts.json'

# write each dict to a new line
with open(path_s, 'w') as outfile:
    for entry in augment_data_bts:
        json.dump(entry, outfile)
        outfile.write('\n')  # Add newline cause PyJSON does not

##Use basic word augmenters


In [ ]:
# using wordnet
augmenter = naw.SynonymAug(aug_p=0.3)  # Swap two adjacent words

# Randomly select an augmenter and apply it to each text in the dataset
augment_data_b = []

# Number of augmentations per item
n = 1

for i in tqdm(range(len(nl_data))):
    for _ in range(n): # Repeat n times
        augmented_text = ' '.join(augmenter.augment(nl_data[i]))
        # Create a new dictionary with the augmented 'nl' text and the same 'code', and append it to combine_data
        augment_data_b.append({'code': code_data[i], 'text': augmented_text})

100%|██████████| 9263/9263 [01:27<00:00, 105.72it/s]


In [ ]:
path_w = '/content/drive/My Drive/dsml/1_project/Dataset/third/wordnet_0.3_1.json'

# write each dict to a new line
with open(path_w, 'w') as outfile:
    for entry in augment_data_b:
        json.dump(entry, outfile)
        outfile.write('\n')  # Add newline cause PyJSON does not

In [ ]:
# Initialize word augmenters
augmenters = [
    naw.RandomWordAug(action="swap", aug_p=0.15),  # Swap two adjacent words
    naw.RandomWordAug(action="delete", aug_p=0.15),  # Randomly remove words
]

# Randomly select an augmenter and apply it to each text in the dataset
augment_data_b = []

# Number of augmentations per item
n = 3

for i in tqdm(range(len(nl_data))):
    for _ in range(n): # Repeat n times
        augmenter = random.choice(augmenters)
        augmented_text = ' '.join(augmenter.augment(nl_data[i]))
        # Create a new dictionary with the augmented 'nl' text and the same 'code', and append it to combine_data
        augment_data_b.append({'code': code_data[i], 'text': augmented_text})

100%|██████████| 9263/9263 [00:16<00:00, 578.68it/s]


In [ ]:
path_4 = '/content/drive/My Drive/dsml/1_project/Dataset/second/4_ranaug_0.15_3.json'

# write each dict to a new line
with open(path_4, 'w') as outfile:
    for entry in augment_data_b:
        json.dump(entry, outfile)
        outfile.write('\n')  # Add newline cause PyJSON does not

In [ ]:
# Initialize word augmenters
augmenters = [
    naw.RandomWordAug(action="swap", aug_p=0.5),  # Swap two adjacent words
    naw.RandomWordAug(action="delete", aug_p=0.5),  # Randomly remove words
]

# Randomly select an augmenter and apply it to each text in the dataset
augment_data_b = []

# Number of augmentations per item
n = 3

for i in tqdm(range(len(nl_data))):
    for _ in range(n): # Repeat n times
        augmenter = random.choice(augmenters)
        augmented_text = ' '.join(augmenter.augment(nl_data[i]))
        # Create a new dictionary with the augmented 'nl' text and the same 'code', and append it to combine_data
        augment_data_b.append({'code': code_data[i], 'text': augmented_text})

100%|██████████| 9263/9263 [00:16<00:00, 570.55it/s]


In [ ]:
path_4 = '/content/drive/My Drive/dsml/1_project/Dataset/second/4_ranaug_0.5_3.json'

# write each dict to a new line
with open(path_4, 'w') as outfile:
    for entry in augment_data_b:
        json.dump(entry, outfile)
        outfile.write('\n')  # Add newline cause PyJSON does not

## Using TF-IDF

In [ ]:
print(nl_data[1:10])

['Check if a number can be represented as sum of two positive perfect cubes | Python3 program for the above approach ; Function to check if N can be represented as sum of two perfect cubes or not ; If it is same return true ; ; If the curr smaller than n increment the lo ; If the curr is greater than curr decrement the hi ; Driver Code ; Function call to check if N can be represented as sum of two perfect cubes or not', 'Generate an N | Python3 program for the above approach ; Function to generate all prime numbers upto 10 ^ 6 ; Initialize sieve [ ] as 1 ; Iterate over the range [ 2 , N ] ; If current element is non - prime ; Make all multiples of i as 0 ; Function to construct an array A [ ] satisfying the given conditions ; Stores the resultant array ; Stores all prime numbers ; Sieve of Erastosthenes ; Append the integer i if it is a prime ; Indicates current position in list of prime numbers ; Traverse the array arr [ ] ; If already filled with another prime number ; If A [ i ] is 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

# Assuming nl_data is your text dataset
vectorizer = TfidfVectorizer()
vectorizer.fit(nl_data)

# Define path to save the model
model_path = '/content/drive/My Drive/dsml/1_project/Dataset/tfidf_model.pkl'

# Save the model
with open(model_path, "wb") as f:
    pickle.dump(vectorizer, f)

In [ ]:
import re
import nlpaug.augmenter.word as naw
import nlpaug.model.word_stats as nmw

def _tokenizer(text, token_pattern=r"(?u)\b\w\w+\b"):
    token_pattern = re.compile(token_pattern)
    return token_pattern.findall(text)

# Instead of loading 20 News Groups, use your nl_data
train_x = nl_data

# Tokenize input
train_x_tokens = [_tokenizer(x) for x in train_x]

# Train TF-IDF model
tfidf_model = nmw.TfIdf()
tfidf_model.train(train_x_tokens)

# Save the model with a clear filename
model_path =  '/content/drive/My Drive/dsml/1_project/Dataset'
tfidf_model.save(model_path)

In [ ]:
# Randomly select an augmenter and apply it to each text in the dataset
augment_data_tf = []

# Number of augmentations per item
n = 1

for i in tqdm(range(len(nl_data))):
    for _ in range(n): # Repeat n times
        augmenter = naw.TfIdfAug(model_path=model_path, tokenizer=_tokenizer,  aug_p=0.15)
        augmented_text = ' '.join(augmenter.augment(nl_data[i]))
        # Create a new dictionary with the augmented 'nl' text and the same 'code', and append it to combine_data
        augment_data_tf.append({'code': code_data[i], 'text': augmented_text})

100%|██████████| 9263/9263 [01:23<00:00, 111.39it/s]


In [ ]:
path_tf = '/content/drive/My Drive/dsml/1_project/Dataset/third/tfidf_0.15_1.json'

# write each dict to a new line
with open(path_tf, 'w') as outfile:
    for entry in augment_data_tf:
        json.dump(entry, outfile)
        outfile.write('\n')  # Add newline cause PyJSON does not